- VITOR GONÇALVES REIS DA SILVA - RA: 156670
- MARCOS VINICIUS GASPAROTO BAUAB - RA: 156717

# NOTAS IMPORTANTES

- Link para nossa base de dados https://www.kaggle.com/datasets/prosperchuks/amazonsatelliteimages
- Utilzamos apenas a pasta train-jpg e train_v2.csv
- O COLAB estava sofrendo para acessar os endereços das imagens de forma rápida sem bugar, então para rodar as 10.000 imagens é necessário baixar as pastas no seu computador

In [ ]:
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
import re
from sklearn.model_selection import KFold
import random


In [ ]:
device = '/device:GPU:0' if tf.test.is_gpu_available() else '/device:CPU:0'

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [ ]:
data_labels = pd.read_csv(r"C:\Users\vitor\OneDrive\Área de Trabalho\train_v2.csv\train_v2.csv")
classes = data_labels['tags'].unique()
print("Quantidade de classes antes da normalização: ", len(classes))

Quantidade de classes antes da normalização:  449


In [ ]:
data_labels.head(10)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
5,train_5,haze primary water
6,train_6,agriculture clear cultivation primary water
7,train_7,haze primary
8,train_8,agriculture clear cultivation primary
9,train_9,agriculture clear cultivation primary road


In [ ]:
# Estamos mapeando as classes, transformando todas as 449 em 2 floresta ou desmatado

def mapear_classes(tags):
    padrao = r"agriculture|mine|habitation|road" # Adicionei road também
    if re.search(padrao, tags):
        return 'desmatado'
    else:
        return 'floresta'

#estamos transformando todas as tags que aparecem cloudy em apenas cloudy para droparmos essas imagens pois são imagens que contém nuvens e não ajudam muito

def retirarImagensImprecisas(tags):
    padrao = r"cloudy|partly_cloudy"
    if re.search(padrao, tags):
        return 'cloudy'
    else:
        return tags

In [ ]:
data_labels["tags"]=data_labels["tags"].apply(retirarImagensImprecisas)
data_labels = data_labels.drop(data_labels[data_labels['tags'] == 'cloudy'].index)  #dropando imagens com nuvem
data_labels["tags"]=data_labels["tags"].apply(mapear_classes) # Floresta ou desmatado
print(data_labels)

        image_name       tags
0          train_0   floresta
1          train_1  desmatado
2          train_2   floresta
3          train_3   floresta
4          train_4  desmatado
...            ...        ...
40472  train_40472  desmatado
40473  train_40473  desmatado
40474  train_40474   floresta
40476  train_40476  desmatado
40477  train_40477  desmatado

[31129 rows x 2 columns]


In [ ]:
classes = data_labels['tags'].unique()
print("Quantidade de classes antes da normalização: ", len(classes))

Quantidade de classes antes da normalização:  2


In [ ]:
data_labels.head(10)

,image_name,tags
0,train_0,floresta
1,train_1,desmatado
2,train_2,floresta
3,train_3,floresta
4,train_4,desmatado
5,train_5,floresta
6,train_6,desmatado
7,train_7,floresta
8,train_8,desmatado
9,train_9,desmatado


In [ ]:
# Definir parâmetros da rede neural
input_shape = (224, 224, 3) #tamanho original das imagens
new_size=(112,112,3) # redução pela metade
num_classes = 2

In [ ]:
# Obter as classes e os nomes das imagens do arquivo CSV
# Aleatorização das imagens pegando do CSV o nome e a tag mas mantendo a ordenação


data_labels_aleatorio = data_labels.sample(n=10000, random_state=42)
classes = data_labels_aleatorio['tags'].tolist()
nomes_imagens= data_labels_aleatorio['image_name'].tolist()

In [ ]:
X = []
y = []

# Diretório onde as imagens estão armazenadas
diretorio_imagens = r"C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg"


# Serve para verificar se não estamos pegando algum endereço que não existe

with tf.device(device):
    for classe, nome_imagem in zip(classes, nomes_imagens):
        caminho_imagem = os.path.join(diretorio_imagens, nome_imagem) + ".jpg"
        print("Caminho da imagem:", caminho_imagem)
        print("Classe:", classe)
        imagem = tf.io.read_file(caminho_imagem)
        imagem = tf.image.decode_image(imagem, channels=3)
        imagem = tf.image.resize(imagem, new_size[:2])
        imagem = tf.expand_dims(imagem, axis=0)
        imagem_array = imagem.numpy()

        X.append(imagem_array)
        y.append(classe)

Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_5030.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_30807.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_23793.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_3850.jpg
Classe: desmatado
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_10674.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_10907.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_21975.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_28843.jpg
Classe: floresta
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\train_14844.jpg
Classe: desmatado
Caminho da imagem: C:\Users\vitor\OneDrive\Área de Trabalho\train-jpg\tra

In [ ]:
# Pré-processamento dos dados

label_encoder = LabelEncoder()
y_enconded = label_encoder.fit_transform(y)
X = np.array(X) / 255.0
y = tf.keras.utils.to_categorical(y_enconded, num_classes)

# K Fold

- Para cada Fold são 4 conjuntos de treino, ou seja 80% dos dados, e 1 conjunto para teste, 20% dos dados. Eles são escolhidos de forma aleatória.

In [ ]:
num_folds = 5  # Número desejado de folds
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Imprimir o nome do fold
    print(f"\033[32mFold {i}:\033[0m")
    print(X_train.shape)
    print(X_test.shape)


Fold 1
(8000, 1, 112, 112, 3)
(2000, 1, 112, 112, 3)
Fold 2
(8000, 1, 112, 112, 3)
(2000, 1, 112, 112, 3)
Fold 3
(8000, 1, 112, 112, 3)
(2000, 1, 112, 112, 3)
Fold 4
(8000, 1, 112, 112, 3)
(2000, 1, 112, 112, 3)
Fold 5
(8000, 1, 112, 112, 3)
(2000, 1, 112, 112, 3)
